In [1]:
import ghmm
import pickle
import numpy as np
from itertools import product
from pprint import pprint
import pysam
import os
import pandas
import re
import editdistance
import sys
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"

In [2]:
NMERS = 3
NSTATES = 4**NMERS

In [3]:
HMM_PARAMS = pickle.load(open("/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/hmm_params.pickle", 'rb'))
HMM_PARAMS = sorted(zip(*[HMM_PARAMS["kmers"], HMM_PARAMS["means"], HMM_PARAMS["stdv"]]))

# Train Model 

In [4]:
def mk_transmat(nmers):
    """make a transition matrix assuming single base steps"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 1/4. if from_kmer[-2:] == to_kmer[:2] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [5]:
# example code for a continuous HMM with gaussian emissions


F = ghmm.Float()  # emission domain of this model

A = mk_transmat(NMERS)
B = [[x[1], x[2]] for x in HMM_PARAMS]   # parameters of emission distributions in pairs of (mu, sigma)
pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

# generate model from parameters
model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)

In [6]:
s = str(model)
print(s)

GaussianEmissionHMM(N=64)
  state 0 (initial=0.02, mu=70.17, sigma=3.84)
    Transitions: ->0 (0.25), ->1 (0.25), ->2 (0.25), ->3 (0.25)
  state 1 (initial=0.02, mu=63.14, sigma=3.47)
    Transitions: ->4 (0.25), ->5 (0.25), ->6 (0.25), ->7 (0.25)

  ...

  state 62 (initial=0.02, mu=53.52, sigma=2.85)
    Transitions: ->56 (0.25), ->57 (0.25), ->58 (0.25), ->59 (0.25)
  state 63 (initial=0.02, mu=49.86, sigma=2.78)
    Transitions: ->60 (0.25), ->61 (0.25), ->62 (0.25), ->63 (0.25)



In [7]:
def result_to_seq(result):
    states = result[0]
    all_kmers = [x[0] for x in HMM_PARAMS]
    kmers = [all_kmers[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [8]:
def predict(means):
    seq = ghmm.EmissionSequence(F, means)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [9]:
predict([44.2, 44.3, 56, 58.2, 56.2, 58.1, 58.2, 60, 30.2])

u'TTTGTGTGT'

# Validate Model 

In [10]:
args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
    "ncores": 4
}

In [11]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline


In [12]:
assert os.path.isfile(args["events"])

In [13]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>mmusMT_PCR1']
GTTAATGTAGCTTAATAACAAAGCAAAGCACTGAAAATGCTTAGATGGATAATTGTATCCCATAAACACAAAGGTTTGGTCCTGGCCTTATAATTAATTA


In [14]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [15]:
prepare_filemap(file_data)

### Make Alignment

In [16]:
fastq_file = "{0}.fastq".format(args["out_basename"])
mk_fastq(fastq_file, file_data)

In [17]:
sam_file = "{0}.sam".format(args["out_basename"])
graphmap(ref_file, fastq_file, sam_file, args["ncores"])

[Index 12:19:11] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 12:19:11] Index already exists. Loading from file.
[Index 12:19:11] Secondary index already exists. Loading from file.
[Index 12:19:11] Index loaded in 0.15 sec.
[Index 12:19:11] Memory consumption: [currentRSS = 513 MB, peakRSS = 513 MB]

[Run 12:19:11] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 12:19:11] Reference genome is assumed to be linear.
[Run 12:19:11] Only one alignment will be reported per mapped read.
[ProcessReads 12:19:11] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 12:19:11] Batch of 81 reads (0 MiB) loaded in 0.00 sec. (16442488 bases)
[ProcessReads 12:19:11] Memory consumption: [currentRSS = 514 MB, peakRSS = 514 MB]
[ProcessReads 12:19:11] Using 4 threads.
[ProcessReads 12:19:12] [CPU time: 3.40 sec, RSS: 537 MB] Read: 81/81 (100.00%) [m: 80, u: 1]                     

In [18]:
bam_file = prepare_sam(args["out_basename"])

[samopen] SAM header is present: 1 sequences.


In [19]:
samfile = pysam.AlignmentFile(bam_file)
reads  = [x for x in samfile.fetch()]
len(reads)

80

In [23]:
def basecall_read(params):
    read_name, ref_pos = params
    file_id, channel_id = get_file_and_channel(read_name)
    print(file_id, channel_id, read_name)
    sys.stdout.flush()
    
    file_obj = get_file(channel_id, file_id)
      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    metrichor_seq = file_obj["fastq"].split("\n")[1]
    ref_seq = "".join([ref[x] for x in ref_pos])
    called_seq = predict(events)
    
    stats = {
        "d_metrichor": int(editdistance.eval(ref_seq, metrichor_seq)),
        "d_caller": int(editdistance.eval(ref_seq, called_seq)), 
        "length": len(ref_seq),
    }

    return (read_name, called_seq, stats)

In [24]:
p = Pool(args["ncores"])
## one can not access a read in parallel (deadlock for whatever reason)
## therfore prepare input parameters outside of map
input_params = [(read.query_name, read.get_reference_positions()) for read in reads]
try:
    results = p.map(basecall_read, input_params)
except KeyboardInterrupt:
    p.terminate()

(10, 132, 'ch132_file10_read')
(9, 215, 'ch215_file9_read')
(17, 132, 'ch132_file17_read')
(16, 215, 'ch215_file16_read')
(35, 132, 'ch132_file35_read')
(1, 203, 'ch203_file1_read')
(20, 211, 'ch211_file20_read')
(12, 206, 'ch206_file12_read')
(11, 133, 'ch133_file11_read')
(28, 211, 'ch211_file28_read')
(25, 215, 'ch215_file25_read')
(11, 156, 'ch156_file11_read')
(1, 157, 'ch157_file1_read')
(0, 135, 'ch135_file0_read')
(5, 227, 'ch227_file5_read')
(10, 156, 'ch156_file10_read')
(12, 211, 'ch211_file12_read')
(21, 212, 'ch212_file21_read')
(3, 211, 'ch211_file3_read')
(19, 135, 'ch135_file19_read')
(12, 141, 'ch141_file12_read')
(18, 142, 'ch142_file18_read')
(14, 215, 'ch215_file14_read')
(8, 141, 'ch141_file8_read')
(3, 195, 'ch195_file3_read')
(15, 215, 'ch215_file15_read')
(13, 141, 'ch141_file13_read')
(10, 215, 'ch215_file10_read')
(24, 212, 'ch212_file24_read')
(27, 212, 'ch212_file27_read')
(6, 156, 'ch156_file6_read')
(3, 156, 'ch156_file3_read')
(0, 157, 'ch157_file0_read')

### Stats

In [25]:
headers, seqs, stats = zip(*results)

In [26]:
stats = pandas.DataFrame(list(stats))

In [27]:
stats.sum(0)

d_caller       226446
d_metrichor    172867
length         387166
dtype: int64

In [28]:
fasta_file_called = "{0}.called.fa".format(args["out_basename"])
with open(fasta_file_called, 'w') as f: 
    for header, seq in zip(headers, seqs): 
        f.write(">" + header + "\n")
        f.write(seq + "\n")

In [29]:
sam_file_called = "{0}.called.sam".format(args["out_basename"])
graphmap(ref_file, fasta_file_called, sam_file_called, args["ncores"])

[Index 12:20:19] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 12:20:19] Index already exists. Loading from file.
[Index 12:20:19] Secondary index already exists. Loading from file.
[Index 12:20:19] Index loaded in 0.15 sec.
[Index 12:20:19] Memory consumption: [currentRSS = 513 MB, peakRSS = 513 MB]

[Run 12:20:19] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 12:20:19] Reference genome is assumed to be linear.
[Run 12:20:19] Only one alignment will be reported per mapped read.
[ProcessReads 12:20:19] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 12:20:19] Batch of 80 reads (0 MiB) loaded in 0.00 sec. (31380616 bases)
[ProcessReads 12:20:19] Memory consumption: [currentRSS = 514 MB, peakRSS = 514 MB]
[ProcessReads 12:20:19] Using 4 threads.
[ProcessReads 12:20:20] [CPU time: 3.40 sec, RSS: 529 MB] Read: 80/80 (100.00%) [m: 79, u: 1]                     

In [30]:
prepare_sam("{0}.called".format(args["out_basename"]))

[samopen] SAM header is present: 1 sequences.


'/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling.called.sorted.bam'